# This notebook will be mainly used for the Capstone Project.

### Part 1: Peer-graded Assignment: Capstone Intro Notebook

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Part 2: Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

#### Setting up all the libraries

In [26]:
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge ProgressBar2 --yes
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
from progressbar import ProgressBar
from bs4 import BeautifulSoup as bts # library for web scraping
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analysis
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests # library to handle requests
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import matplotlib as mp # library for visualization
import folium # map rendering library
import lxml
import re
from time import sleep
%matplotlib inline

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages 

#### Scrape the Toronto Wikipedia page and wrangle the data

#### Creating a beautifulsoup4 object from the Toronto Wikipedia page

In [4]:
toronto_source  = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [5]:
toronto_soup = bts(toronto_source,'lxml')

#### Find all tabulated data from the text source

In [6]:
toronto_table = toronto_soup.findAll('table')

#### Removing citations and comments from scarped data

In [7]:
toronto_table_text = toronto_table[0].tbody.text
toronto_table_text = re.sub("\[.*?\]", "", toronto_table_text)
toronto_table_text = re.sub("\(.*?\)", "", toronto_table_text)

#### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [8]:
# Splitting the string into a list at every new line, I have renamed the Postcode to PostalCode and Neighbourhood to Neighborhood as per the assignment
toronto_table_list = toronto_table_text.split('\n')
toronto_table_list[1] = 'PostalCode'
toronto_table_list[3] = 'Neighborhood'
del toronto_table_list[-1]

#### Creating the columns for the table

In [9]:
toronto_table_columns = toronto_table_list[0:5]

#### Reshaping the data into a 2D NumPy array

In [10]:
if (len(toronto_table_list[5:]) % 5 == 0):
    toronto_table_data = np.array(toronto_table_list[5:]).reshape(len(toronto_table_list[5:]) // 5,5)
else:
    print("Number of table elements is incorrect!")

#### Creating a Pandas DataFrame from the table

In [11]:
toronto_dataframe =  pd.DataFrame(np.nan_to_num(toronto_table_data),columns = toronto_table_columns)

In [12]:
# Displaying the top twelve rows of the DataFrame
print(toronto_dataframe.shape)
toronto_dataframe.head()

(288, 5)


,,PostalCode,Borough,Neighborhood,
0,,M1A,Not assigned,Not assigned,
1,,M2A,Not assigned,Not assigned,
2,,M3A,North York,Parkwoods,
3,,M4A,North York,Victoria Village,
4,,M5A,Downtown Toronto,Harbourfront,


#### Dropping cells where Borough is equal to Not assigned

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [13]:
toronto_dataframe.drop( toronto_dataframe[ toronto_dataframe['Borough'] == 'Not assigned' ].index , inplace=True)
print(toronto_dataframe.shape)
toronto_dataframe.head()

(211, 5)


,,PostalCode,Borough,Neighborhood,
2,,M3A,North York,Parkwoods,
3,,M4A,North York,Victoria Village,
4,,M5A,Downtown Toronto,Harbourfront,
5,,M5A,Downtown Toronto,Regent Park,
6,,M6A,North York,Lawrence Heights,


More than one neighborhood exist for one postal code area. For example, you will notice that M1B is listed twice and has two neighborhoods: Rouge and Malvern. These two rows is combined into one row with the neighborhoods separated with a comma as shown in first row in the below dataframe.

In [14]:
toronto_dataframe = toronto_dataframe.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda Neighborhood: ', '.join(Neighborhood)).to_frame()
toronto_dataframe.reset_index(inplace=True)
print(toronto_dataframe.shape)
toronto_dataframe.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows have been combined into one row with the neighborhoods separated with a comma. 

In [15]:
print(toronto_dataframe.loc[toronto_dataframe['PostalCode'] == 'M5A'])

   PostalCode           Borough               Neighborhood
53        M5A  Downtown Toronto  Harbourfront, Regent Park


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

#### Notice for postalcode M7A the value of the Borough and the Neighborhood columns is Queen's Park.

In [16]:
toronto_dataframe.loc[toronto_dataframe.Neighborhood == 'Not assigned', 'Neighborhood'] = toronto_dataframe.Borough
print(toronto_dataframe.loc[toronto_dataframe['PostalCode'] == 'M7A'])

   PostalCode       Borough  Neighborhood
85        M7A  Queen's Park  Queen's Park


#### Using the .shape method to print the number of rows of your toronto_dataframe.

In [17]:
toronto_dataframe.shape

(103, 3)

Added two columns for populating Latitude and Longitude for each Postal Code

In [18]:
toronto_dataframe['Latitude'] = np.nan
toronto_dataframe['Longitude'] = np.nan
toronto_dataframe.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",NaN,NaN
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",NaN,NaN
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",NaN,NaN
3,M1G,Scarborough,Woburn,NaN,NaN
4,M1H,Scarborough,Cedarbrae,NaN,NaN


In [19]:
pbar = ProgressBar()
geolocator = Nominatim()
for index in pbar(range(0,toronto_dataframe['PostalCode'].shape[0])):
    address = toronto_dataframe.loc[index,'PostalCode'] + ", Ontario"
    location = geolocator.geocode(address, timeout = None)
    if (location != None):
        toronto_dataframe.loc[index,'Latitude'] = location.latitude
        toronto_dataframe.loc[index,'Longitude'] = location.longitude
    sleep(1)
print(toronto_dataframe.shape)
toronto_dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  
N/A% (0 of 103) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `timeout=None` has been passed to a geocoder call. Using default geocoder timeout. In geopy 2.0 the behavior will be different: None will mean "no timeout" instead of "default geocoder timeout". Pass geopy.geocoders.base.DEFAULT_SENTINEL instead

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",34.065846,-117.64843
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",34.065846,-117.64843
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",NaN,NaN
3,M1G,Scarborough,Woburn,NaN,NaN
4,M1H,Scarborough,Cedarbrae,NaN,NaN


#### If the above calls fails to populate the latitiude and longitude columns, I can still use the below csv file of geographical coordinates to populate those columns.

In [20]:
file_name='http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
tor_coordinates=pd.read_csv(file_name)
print(tor_coordinates.shape)
tor_coordinates.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Populating the Latitude and Longitude columns from the tor_coordinates dataframe

In [21]:
for index in range(0,toronto_dataframe['PostalCode'].shape[0]):
    postal_indexes = tor_coordinates.loc[tor_coordinates['Postal Code'] == toronto_dataframe['PostalCode'].iloc[index]]
    if (postal_indexes.index.size != 0):
        toronto_dataframe.loc[index, 'Latitude'] = tor_coordinates['Latitude'].iloc[postal_indexes.index[0]]
        toronto_dataframe.loc[index, 'Longitude'] = tor_coordinates['Longitude'].iloc[postal_indexes.index[0]]

In [22]:
print(toronto_dataframe.shape)
toronto_dataframe.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Use geopy library to get the latitude and longitude values of Toronto City.

In [23]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of toronto are 43.6523873, -79.3835641.


## Explore and cluster the neighborhoods in Toronto

### All Toronto neighborhoods with boroughs that contain the word Toronto 

Slice the original dataframe toronto_dataframe and create a new dataframe tor_borough of all boroughs that contain the word Toronto.

In [24]:
tor_borough = toronto_dataframe[toronto_dataframe['Borough'].str.contains('Toronto')].reset_index(drop=True)
print(tor_borough.shape)
tor_borough.head(38)

(38, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


### Create a map of Toronto with neighborhoods superimposed on top where Boroughs that contain the word Toronto.

In [25]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(tor_borough['Latitude'], tor_borough['Longitude'], tor_borough['Borough'], tor_borough['Neighborhood']):
    label = '{}, {}'.format(borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto